In [101]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import torch

In [102]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/sudoghut/contradictory-my-dear-watson/main/data/train_10.csv')
print(df_raw.head(5))
print(df_raw["label"].value_counts())


           id                                            premise  \
0  5130fd2cb5  and these comments were considered in formulat...   
1  5b72532a0b  These are issues that we wrestle with in pract...   
2  3931fbe82a  Des petites choses comme celles-là font une di...   
3  5622f0c60b  you know they can't really defend themselves l...   
4  86aaa48b45  ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...   

                                          hypothesis lang_abv language  label  
0  The rules developed in the interim were put to...       en  English      0  
1  Practice groups are not permitted to work on t...       en  English      2  
2              J'essayais d'accomplir quelque chose.       fr   French      0  
3  They can't defend themselves because of their ...       en  English      0  
4    เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร       th     Thai      1  
0    5
2    2
1    2
Name: label, dtype: int64


In [103]:
df_raw["premise"]

0    and these comments were considered in formulat...
1    These are issues that we wrestle with in pract...
2    Des petites choses comme celles-là font une di...
3    you know they can't really defend themselves l...
4    ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...
5    Bir çiftlikte birisinin, ağıla kapatılmış bu ö...
6    ریاست ہائے متحدہ امریکہ واپس آنے پر، ہج ایف بی...
7                From Cockpit Country to St. Ann's Bay
8    Look, it's your skin, but you're going to be i...
Name: premise, dtype: object

In [104]:
# tokenizer: https://huggingface.co/bert-base-multilingual-cased
# tokenizer idea from: https://www.kaggle.com/code/anasofiauzsoy/tutorial-notebook/notebook

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
# text = df_raw["premise"][4]
# encoded_input = tokenizer(text, return_tensors='pt')
# print(df_raw["premise"][4])
# print(encoded_input)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [107]:
# df_raw["premise_embedded"] = tokenizer(df_raw["premise"], return_tensors='pt')

df_raw["data_embedded"] = df_raw.apply(lambda row: tokenizer(row['premise']+row['hypothesis']), axis=1)
# df_raw["hypothesis_embedded"] = df_raw.apply(lambda row: tokenizer(row['hypothesis'])["input_ids"], axis=1)
# df_raw["data_embedded"] = df_raw["premise_embedded"] + df_raw["hypothesis_embedded"]
# df_raw["data_embedded"] = df_raw["data_embedded"]
print(df_raw["data_embedded"][:3])

0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
Name: data_embedded, dtype: object


In [99]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split([df_raw[["data_embedded"]]], 
                                                    df_raw[["label"]], 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible
print(X_train[:5])
# print(X_train[:3])
# X_train.drop([0], axis=1,inplace=True)
# y_train.drop(axis=1,inplace=True)

ValueError: blocks must be 2-D

In [67]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


X = 
[
	X_train[""]
]
).tocsr()

In [92]:
evals_result = {}

# params = {
#     'objective': 'mse',
#     'metric': 'rmse',
#     'num_leaves': 2 ** 7 - 1,
#     'learning_rate': 0.005,
#     'feature_fraction': 0.75,
#     'bagging_fraction': 0.75,
#     'bagging_freq': 5,
#     'seed': 1,
#     'verbose': 1
# }
params = {
          'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 150,
          'max_depth': -1,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.01,
          'min_child_samples': 150,
          'min_child_weight': 0.1,
          'verbosity': -1,
          'data_random_seed': 3,
          'early_stop': 100,
          'verbose_eval': 100,
          'num_rounds': 3000}

feature_name = X_train.columns.tolist()

feature_name_indexes = ['data_embedded']

lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_eval = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

evals_result = {}
gbm = lgb.train(
        params, 
        lgb_train,
        num_boost_round=3000,
        valid_sets=(lgb_train, lgb_eval), 
        feature_name = feature_name,
        categorical_feature = feature_name_indexes,
        verbose_eval=5, 
        evals_result = evals_result,
        early_stopping_rounds = 100)

c:\Users\sudos\anaconda3\envs\pytorch\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
c:\Users\sudos\anaconda3\envs\pytorch\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['data_embedded']
  _log_warning('categorical_feature in Dataset is overridden.\n'


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: data_embedded

In [ ]:
# gbm = lgb.train(
#         params, 
#         lgb_train,
#         num_boost_round=3000,
#         valid_sets=(lgb_train, lgb_eval), 
#         verbose_eval=5, 
#         evals_result = evals_result,
#         early_stopping_rounds = 100)

TypeError: Cannot initialize Dataset from Series